In [17]:
import os
import re
import shutil
import time
from pathlib import Path

from dotenv import load_dotenv
from github import Github
import pandas as pd
import requests

load_dotenv()

github = Github(os.getenv('GITHUB_TOKEN'))
repo   = github.get_repo("AlejandroDiBattista/tup25-p4")

def leer(origen: str = "./alumnos.md") -> list:
    """
    Lee un archivo markdown con datos de alumnos y devuelve una lista de diccionarios.
    """

    print(f"> Leyendo {origen}...")
    salida, comision = [], ""
    with open(origen, "r", encoding='utf-8') as archivo:
        for linea in archivo:
            if not linea.strip():
                continue

            # Normalizar espacios iniciales para aceptar tanto '- ...' como ' - ...'
            sline = linea.lstrip()

            if sline.startswith("##"):
                # Tomar último caracter numérico de la cabecera (p. ej. '## Comisión 1')
                comision = "C" + sline.strip("# \n")[-1]

            if sline.startswith("- "):
                sline = sline.replace("@","")
                partes = re.split(r"\s{2,}", sline.strip("- \n@"))
                if len(partes) < 5:
                    partes.append("")
                legajo, nombre, telefono, practicos, github = partes
                valores = [legajo, nombre, telefono, github, comision]
                valores = dict(zip('legajo nombre telefono github comision'.split(), valores))

                for i, p in enumerate(practicos, 1):
                    if p == "0": p = "🔴"
                    if p == "1": p = "🟢"
                    valores[f"TP{i}"] = p

                salida.append(valores)
    print(f"| {len(salida)} alumnos leídos.")
    return sorted(salida, key=lambda x: [x['comision'], x['legajo']])


def escribir(alumnos, destino="alumnos2.md", orden_practicos=False):
    """
    Escribe un archivo markdown con datos de alumnos a partir de una lista de diccionarios.
    """
    def orden_estado(alumno):
        if all(alumno[p] == "🔴" for p in practicos):
            return 100
        for i, p in enumerate(practicos, 1):
            if alumno[p] == "🔴":
                return i
        return 99
    
    def normalizar_presentados(practicos):
        salida = []
        for p in practicos:
            if p == "0": p = "🔴"
            if p == "⚪": p = "🔴"
            if p == "1": p = "🟢"
            salida.append(p)
        return "".join(salida)
    
    practicos = [f"TP{i}" for i in range(1, 20)]
    practicos = [p for p in practicos if p in alumnos[0]]

    print(f"> Escribiendo {destino}...")

    if orden_practicos:
        alumnos = sorted(alumnos, key=lambda alumno: [alumno['comision'], orden_estado(alumno), alumno["nombre"]])
    else:
        alumnos = sorted(alumnos, key=lambda alumno: [alumno['comision'], alumno["nombre"]])
        
    with open(destino, "w", encoding='utf-8') as archivo:
        archivo.write("# Programación 4 | TUP 25\n")
        for comision in sorted(set(alumno['comision'] for alumno in alumnos)):
            archivo.write(f"\n## Comisión {comision[-1]}\n\n")
            archivo.write("```text\n")
            for alumno in alumnos:
                presentado = map(str, [alumno[p] for p in practicos])
                presentado = normalizar_presentados(presentado)
                github = f"@{alumno['github']}" if len(alumno['github']) else ""
                if alumno['comision'] == comision:
                    archivo.write(f"- {alumno['legajo']}  {alumno['nombre']:40}  {alumno['telefono']}  {presentado}  {github:20}\n")
            archivo.write("```\n")
    print(f"| {len(alumnos)} alumnos guardados")
    return alumnos


def convertir_vcard(alumnos, archivo_destino="contactos_alumnos.vcf"):
    """
    Convierte una lista de alumnos en un archivo VCard compatible con WhatsApp.
    """

    print(f"> Generando archivo VCard: {archivo_destino}...")
    
    with open(archivo_destino, "w", encoding='utf-8') as archivo:
        for alumno in alumnos:
            if alumno['github'].strip() == "": continue
            nombre_completo = alumno['nombre'].strip()
            partes_nombre = nombre_completo.split(', ')
            
            if len(partes_nombre) == 2:
                apellidos = partes_nombre[0].strip()
                nombres = partes_nombre[1].strip()
            else:
                nombres = nombre_completo
                apellidos = ""
            
            telefono = re.sub(r'[^\d+]', '', alumno['telefono'])
            
            vcard = [
                "BEGIN:VCARD",
                "VERSION:3.0",
                f"FN:{nombre_completo}",  # Nombre completo para mostrar
                f"N:{apellidos};{nombres};;;",  # Apellido;Nombre;SegundoNombre;Prefijo;Sufijo
                f"TEL;TYPE=CELL:{telefono}",  # Teléfono celular
                f"NOTE:Legajo: {alumno['legajo']} - Comision: P4-{alumno['comision']} - Github: {alumno['github']}",  # Nota con legajo y comisión
                "END:VCARD"
            ]
            
            # Escribir el VCard al archivo
            archivo.write("\n".join(vcard) + "\n\n")

    print(f"| Archivo generado con {len(alumnos)} contactos.")
    return len(alumnos)


def normalizar(origen='alumnos.md', orden='nombre'):
    """ Lee un archivo markdown con datos de alumnos, los ordena y escribe en otro archivo.
        - orden: `nombre` | `legajo`
    """
    alumnos = leer(origen)
    if orden!="practicos":
        alumnos = sorted(alumnos, key=lambda x: [x['comision'], x[orden]])
    return escribir(alumnos, origen, orden_practicos= orden=="practicos")


def carpeta(alumno, tp=None): 
    """ Calcula el nombre de la carpeta para un alumno. """
    return f"../tp/{alumno['legajo']} - {alumno['nombre']}{'' if tp is None else f'/tp{tp}'}"


def crear_carpetas(alumnos):
    """Crea carpetas para cada alumno (tp/<legajo> - <nombre>) y un archivo info.txt dentro.
    
    Si existe una carpeta con el mismo legajo pero nombre diferente, la renombra.
    Si no existe carpeta, la crea nueva.
    """
    raiz = Path('../tp')  # Usar la ruta correcta hacia tp/
    raiz.mkdir(parents=True, exist_ok=True)  # Asegurar que el directorio tp existe
    
    print(f"> Procesando {len(alumnos)} alumnos en {raiz}...")
    for a in alumnos:
        # Calcular nombre de carpeta destino (sin el prefijo ../tp/)
        destino = raiz / f"{a['legajo']} - {a['nombre']}"
        
        # Buscar carpetas existentes con el mismo legajo
        carpetas_existentes = list(raiz.glob(f"{a['legajo']} -*"))

        if carpetas_existentes:
            # Si existe al menos una carpeta con este legajo
            carpeta_actual = carpetas_existentes[0]  # Tomar la primera
            
            if carpeta_actual != destino:
                # Si el nombre es diferente, renombrar
                print(f"  Renombrando: {carpeta_actual} \n             > {destino}")
                try:
                    carpeta_actual.rename(destino)
                except Exception as e:
                    print(f"Error al renombrar {carpeta_actual}: {e}")
                    continue
        else:
            # Si no existe carpeta con este legajo, crearla
            print(f"Creando nueva carpeta: {destino}")
            destino.mkdir(parents=True, exist_ok=True)

        # Asegurar que la carpeta existe y escribir/actualizar info.txt
        info_file = destino / "info.txt"
        if not info_file.exists():
            info_content = (
                f"Legajo: {a['legajo']}\n"
                f"Nombre: {a['nombre']}\n"
                f"Teléfono: {a['telefono']}\n"
                f"Comisión: {a['comision']}\n"
            )
            info_file.write_text(info_content, encoding='utf-8')
            print(f"Creado info.txt en: {destino}")


def normalizar_carpetas():
    """ Normaliza el archivo y crea las carpetas correspondientes. """
    alumnos = leer()
    print("> Normalizando nombres de carpetas...")
    crear_carpetas(alumnos)
    print("| Proceso normalización de carpetas finalizado.")

alumnos = leer()
normalizar(orden="practicos")
normalizar_carpetas()

# crear_carpetas(leer())

> Leyendo ./alumnos.md...
| 82 alumnos leídos.
> Leyendo alumnos.md...
| 82 alumnos leídos.
> Escribiendo alumnos.md...
| 82 alumnos guardados
> Leyendo ./alumnos.md...
| 82 alumnos leídos.
> Normalizando nombres de carpetas...
> Procesando 82 alumnos en ../tp...
  Renombrando: ../tp/61200 - Gauna Serrano, Martin Javier 
             > ../tp/61200 - Gauna Serrano, Martín Javier
Error al renombrar ../tp/61200 - Gauna Serrano, Martin Javier: [Errno 66] Directory not empty: '../tp/61200 - Gauna Serrano, Martin Javier' -> '../tp/61200 - Gauna Serrano, Martín Javier'
  Renombrando: ../tp/61627 - Juarez Fernandez, Lourdes Abril 
             > ../tp/61627 - Juárez Fernández, Lourdes Abril
Error al renombrar ../tp/61627 - Juarez Fernandez, Lourdes Abril: [Errno 66] Directory not empty: '../tp/61627 - Juarez Fernandez, Lourdes Abril' -> '../tp/61627 - Juárez Fernández, Lourdes Abril'
| Proceso normalización de carpetas finalizado.


In [2]:
def copiar_tp(tp=1, alumnos=None, incluir_ocultos=True, follow_symlinks=False):
    """
    Copia recursivamente todos los archivos y subcarpetas de enunciados/tp{tp}
    a la carpeta correspondiente de cada alumno (tp/<legajo> - <nombre>/tp{tp}).

    - incluir_ocultos: si es False, ignora entradas que comienzan con '.' (p. ej. .DS_Store, .git)
    - follow_symlinks: si es True, copia el contenido al que apunta el symlink; si es False, copia el link como archivo
    """
    if alumnos is None:
        alumnos = leer()

    origen_base = Path(f"../enunciados/tp{tp}")
    if not origen_base.exists():
        print(f"No existe {origen_base}")
        return

    total_archivos = 0
    for a in alumnos:
        destino_base = Path(carpeta(alumno=a, tp=tp))  # ya incluye 'tp/.../tp{tp}'
        destino_base.mkdir(parents=True, exist_ok=True)

        # Recorremos el árbol de 'origen_base' y replicamos estructura + archivos
        for root, dirs, files in os.walk(origen_base):
            root_path = Path(root)

            # Opcional: filtrar carpetas/archivos ocultos
            if not incluir_ocultos:
                dirs[:] = [d for d in dirs if not d.startswith('.')]

            rel = root_path.relative_to(origen_base)
            dest_dir = destino_base / rel
            dest_dir.mkdir(parents=True, exist_ok=True)

            for name in files:
                if not incluir_ocultos and name.startswith('.'):  # p. ej. .DS_Store
                    continue
                src = root_path / name
                dst = dest_dir / name
                try:
                    shutil.copy2(src, dst, follow_symlinks=follow_symlinks)
                    total_archivos += 1
                except Exception as e:
                    print(f"Error copiando {src} -> {dst}: {e}")

    print(f"| Copia finalizada. {len(alumnos)} alumnos, {total_archivos} archivos copiados en total.")


alumnos = leer()
# crear_carpetas(alumnos)
# copiar_tp(3, alumnos)

> Leyendo ./alumnos.md...
| 81 alumnos leídos.


In [3]:
# Ejemplo de uso de la función convertir_vcard
alumnos = leer('alumnos.md')
print(f"Se encontraron {len(alumnos)} alumnos")

# Generar archivo VCard con todos los alumnos
contactos_generados = convertir_vcard(alumnos, "alumnos.vcf")

print("Archivos VCard generados exitosamente!")
print("Puedes importar estos archivos directamente en WhatsApp o en tu aplicación de contactos.")

> Leyendo alumnos.md...
| 81 alumnos leídos.
Se encontraron 81 alumnos
> Generando archivo VCard: alumnos.vcf...
| Archivo generado con 81 contactos.
Archivos VCard generados exitosamente!
Puedes importar estos archivos directamente en WhatsApp o en tu aplicación de contactos.


In [16]:
# Gestion de repositorio y Pull Requests

def normalizar_titulo(titulo, tp, alumnos):
    if match:= re.search(r'(\d{5})', titulo):
        legajo = match.group(1)
        nombre_alumno = next((a['nombre'] for a in alumnos if a['legajo'] == legajo), "Desconocido")
        return f"TP{tp} - {legajo} - {nombre_alumno}"
    else:
        return titulo

def cargar_github_user(alumnos):
    print(f"> Cargando usuarios de GitHub en {len(alumnos)} alumnos...")

    practicos = set()
    prs = repo.get_pulls(state='closed', sort='created', base='main')
    for pr in prs:
        tp, legajo, *_ = pr.title.split(" - ")
        practicos.add(tp)
        if alumno := next((x for x in alumnos if x['legajo'].strip() == legajo), None):
            alumno["github"] = pr.user.login
            alumno[tp] = 1

    for a in alumnos:
        for c in sorted(practicos):
            if c not in a: 
                a[c] = 0

    print(f"|")
    return alumnos


def normalizar_titulo_pr(tp=3, estado='open', alumnos=None, min=0, max=9999):
    print(f"> Pull Requests pendientes:")

    open_pulls = repo.get_pulls(state=estado, sort='created', base='main')
    for pr in open_pulls:
        if not (min <= pr.number <= max): continue

        pr = repo.get_pull(pr.number)  # Refrescar datos del PR
        nuevo = normalizar_titulo(pr.title, tp, alumnos)
        if pr.title != nuevo:
            print(f"+ #{pr.number:3}: {nuevo} (por {pr.user.login}) {pr.state}{"*" if pr.title != nuevo else ""}")
            print(f"    >>: {pr.title}")
            pr.edit(title=nuevo)
        else:
            print(f"- #{pr.number:3}: {nuevo} (por {pr.user.login}) {pr.state}")

    cols = sorted({c for a in alumnos for c in a.keys() if c.startswith("TP")})

    print(f"| fin {cols}")


def aceptar_prs_sin_conflictos():
    open_pulls = repo.get_pulls(state='open', sort='created', base='main')
    print("> Intentando merge de PRs sin conflictos:")
    for pr in open_pulls:
        pr = repo.get_pull(pr.number)       # Refrescar datos
        if pr.mergeable is None:
            time.sleep(5)
            pr = repo.get_pull(pr.number)   # Refrescar
        if pr.mergeable:
            try:
                pr.merge()
                print(f"- ✅ PR #{pr.number} mergeado: {pr.title}")
            except Exception as e:
                print(f"- ❌ Error al mergear PR #{pr.number}: {e}")
        elif pr.mergeable is False:
            print(f"- 👁️ PR #{pr.number} tiene conflictos: {pr.title}")
        else:
            print(f"- ⏳ Tiempo agotado para PR #{pr.number}: {pr.title}")
    print("| fin")

# Funciones
# 1. normalizar_pr -> renombra los PRs abiertos con formato estandar
# 2. merge_prs_sin_conflictos -> intenta mergear los PRs sin conflictos

alumnos = leer()
# normalizar_titulo_pr(tp=3, estado='open', alumnos=alumnos, min=65)
cargar_github_user(alumnos)
escribir(alumnos, "alumnos.md")
# aceptar_prs_sin_conflictos()


# Cargar lista de alumnos en un DataFrame y mostrarlo como tabla
# df_alumnos = pd.DataFrame(alumnos)

# df_alumnos

> Leyendo ./alumnos.md...
| 82 alumnos leídos.
> Cargando usuarios de GitHub en 82 alumnos...
|
> Escribiendo alumnos.md...
| 82 alumnos guardados


[{'legajo': '61131',
  'nombre': 'Agostino Colombres, Juan Manuel',
  'telefono': '(381) 684-8399',
  'github': 'JuanmaAgostino',
  'comision': 'C1',
  'TP1': 1,
  'TP2': 1,
  'TP3': 1},
 {'legajo': '62171',
  'nombre': 'Ballespin, Cristián',
  'telefono': '(381) 604-5734',
  'github': '',
  'comision': 'C1',
  'TP1': '🔴',
  'TP2': '🔴',
  'TP3': '🔴'},
 {'legajo': '99001',
  'nombre': 'Branda, César',
  'telefono': '(381) 212-6932',
  'github': '',
  'comision': 'C1',
  'TP1': '🔴',
  'TP2': '🔴',
  'TP3': '🔴'},
 {'legajo': '61565',
  'nombre': 'Carrizo, Mauro',
  'telefono': '(381) 610-8745',
  'github': 'DarkoDarkShadow',
  'comision': 'C1',
  'TP1': 1,
  'TP2': 1,
  'TP3': 1},
 {'legajo': '61236',
  'nombre': 'Drachenberg, Franco Eduardo',
  'telefono': '(381) 670-8708',
  'github': 'FDrach',
  'comision': 'C1',
  'TP1': 1,
  'TP2': 1,
  'TP3': 1},
 {'legajo': '61551',
  'nombre': 'Díaz Valdez, Ignacio',
  'telefono': '(381) 695-8566',
  'github': 'IgnacioDiazV',
  'comision': 'C1',
  

In [6]:

def descargar_avatar_github(username: str, destino: str | Path = "./avatars", nombre_archivo: str | None = None, token: str | None = None, use_direct_url: bool = True) -> Path:
    destino = Path(destino)
    destino.mkdir(parents=True, exist_ok=True)

    base = nombre_archivo if nombre_archivo else username

    def _guardar(img_resp: requests.Response, ruta_base: Path) -> Path:
        # Elegir extensión por Content-Type
        content_type = img_resp.headers.get("Content-Type", "").lower()
        _ext = ".jpg"
        if "png" in content_type:
            _ext = ".png"
        elif any(x in content_type for x in ["jpeg", "jpg"]):
            _ext = ".jpg"
        elif "gif" in content_type:
            _ext = ".gif"
        destino_archivo = ruta_base.with_suffix(_ext)
        with open(destino_archivo, "wb") as f:
            for chunk in img_resp.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
        print(f"Avatar guardado en: {destino_archivo}")
        return destino_archivo

    # 1) Intentar descargar directamente desde la URL pública del avatar (sin consumir API core)
    if use_direct_url:
        direct_url = f"https://github.com/{username}.png"
        try:
            img_resp = requests.get(direct_url, stream=True, timeout=30)
            if img_resp.status_code == 200:
                return _guardar(img_resp, destino / base)
            else:
                print(f"No se pudo usar URL directa ({img_resp.status_code}), se intentará API.")
        except Exception as e:
            print(f"Fallo URL directa: {e}. Se intentará API.")

    # 2) Consultar API para obtener avatar_url (con token si está disponible)
    token = token or os.getenv("GITHUB_TOKEN")
    headers = {"Accept": "application/vnd.github+json"}
    if token:
        headers["Authorization"] = f"Bearer {token}"

    url = f"https://api.github.com/users/{username}"
    resp = requests.get(url, headers=headers, timeout=20)
    if resp.status_code == 403:
        # Rate limit excedido
        reset = resp.headers.get("X-RateLimit-Reset")
        if reset and reset.isdigit():
            wait = max(0, int(reset) - int(time.time()) + 2)
            raise RuntimeError(f"Rate limit excedido. Reintentar en ~{wait}s o usar un token válido en GITHUB_TOKEN.")
        raise RuntimeError("Rate limit excedido. Configure GITHUB_TOKEN para aumentar el límite (hasta 5000/hora).")
    resp.raise_for_status()
    data = resp.json()
    avatar_url = data.get("avatar_url")
    if not avatar_url:
        raise RuntimeError(f"No se encontró avatar para el usuario: {username}")

    img_resp = requests.get(avatar_url, stream=True, timeout=30)
    img_resp.raise_for_status()
    return _guardar(img_resp, destino / base)


def bajar_avatars():
    descargados = []

    alumnos = leer()
    for alumno in alumnos:
        gh = (alumno.get('github') or "").strip("@ ")
        if not gh: continue
        destino = Path(carpeta(alumno))  # carpeta devuelve ../tp/<legajo> - <nombre>
        destino.mkdir(parents=True, exist_ok=True)
        nombre_base = str(alumno['legajo'])
        try:
            ruta = descargar_avatar_github(gh, destino=destino, nombre_archivo=nombre_base, use_direct_url=True)
            descargados.append(ruta)
            time.sleep(0.2)  # pequeño throttle para ser buen ciudadano
        except Exception as e:
            print(f"No se pudo bajar avatar de {gh} ({alumno['legajo']}): {e}")

    print(f"Avatares descargados: {len(descargados)}")
# Ejemplo individual:
bajar_avatars()
# descargar_avatar_github("alejandrodibattista")

> Leyendo ./alumnos.md...
| 81 alumnos leídos.
Avatar guardado en: ../tp/54911 - Jatib, Rodrigo Gabriel/54911.png
Avatar guardado en: ../tp/61033 - Quiroga, José María/61033.png
Avatar guardado en: ../tp/61035 - Ledesma, Paulo Marcelo/61035.png
Avatar guardado en: ../tp/61046 - Ponce, Luz Micaela/61046.jpg
Avatar guardado en: ../tp/61073 - González Chavez, Rodrigo/61073.png
Avatar guardado en: ../tp/61084 - Ramírez, Leonardo/61084.png
Avatar guardado en: ../tp/61115 - Ledesma, Santiago Esteban/61115.png
Avatar guardado en: ../tp/61120 - Guerrero, Ana Sofía/61120.png
Avatar guardado en: ../tp/61131 - Agostino Colombres, Juan Manuel/61131.png
Avatar guardado en: ../tp/61167 - Giacobbe, Franco/61167.jpg
Avatar guardado en: ../tp/61198 - Ferreyra Appas, Santiago Nicolás/61198.jpg
Avatar guardado en: ../tp/61236 - Drachenberg, Franco Eduardo/61236.png
Avatar guardado en: ../tp/61240 - Timo, Héctor Gabriel/61240.png
Avatar guardado en: ../tp/61315 - Robles, Thiago/61315.jpg
Avatar guardado e